In [244]:
from selenium import webdriver
import urllib.request
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import codecs
import re
import pandas as pd
from datetime import datetime

#Generate pictures By P.Tor
import csv
import numpy as np
import pygal
import codecs


In [245]:
pizza_pala_romana_URL = 'https://www.facebook.com/pg/Palapizzaromanabistrot/reviews/?ref=page_internal'
# pizza_pala_romana_URL = 'https://www.facebook.com/pg/cafe.mademoiselle.bangkok.2015/reviews/?ref=page_internal'
usr = "scappingTeam@gmail.com"
pwd = "torn2537"
shopName = 'romana'
loop=20

graphFileList=['romana','mademoiselle']

In [246]:
page = urllib.request.urlopen(pizza_pala_romana_URL).read()
soup = BeautifulSoup(page, 'html.parser')

data = soup.find_all("div", class_="_5pbx userContent _3576")
print(data)

[<div class="_5pbx userContent _3576" data-ft='{"tn":"K"}'><p>Review only for takeaway, as I have never dined in.<br/> Great food at great prices! Service is also fast and professional.<br/> Will come back!</p></div>, <div class="_5pbx userContent _3576" data-ft='{"tn":"K"}'><p>Large selection of pizza, pastas and desserts (wow). Southern Italy recipes not seen anywhere else. Italian beers on draft too. Very popular with locals and expats. Attentive staff. 5 stars!</p></div>, <div class="_5pbx userContent _3576" data-ft='{"tn":"K"}'><p>อย่าไปซื้อพนังงานนิสัยเเย่มาด่าลูกค้าเหี้ย<br/> ผู้จัดการเข้าข้างลูก</p></div>, <div class="_5pbx userContent _3576" data-ft='{"tn":"K"}'><p>Pala pizza 5 Stars <br/> The best Roman pizza ever in Bangkok! fast and effective service. traditional and authentic Italian food !  thank you so much .</p></div>, <div class="_5pbx userContent _3576" data-ft='{"tn":"K"}'><p>ดีทุกอย่าง...อาหาร. รสชาติ. สภานที่. บริการ...<br/> ไม่ต้องอธิบายมาก</p></div>]


In [247]:


# This function will close notification of Chrome browser after log in facebook account.
def handle_notification_Chrome():
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(chrome_options=chrome_options)
    return driver

#Log in to facebook
driver = handle_notification_Chrome()

driver.get('https://www.facebook.com/')
assert "Facebook" in driver.title
elem = driver.find_element_by_id("email")
elem.send_keys(usr)
elem = driver.find_element_by_id("pass")
elem.send_keys(pwd)
elem.send_keys(Keys.RETURN)
print("Log in sucessfully")
#Go to the page.
driver.get(pizza_pala_romana_URL)
#Dump source code of Facebook web page on target url

last_i = 0 
time.sleep(3) 
file_object = codecs.open("Dump_"+shopName+".txt", "w", "utf-8") 
while last_i<loop: 
    last_i = last_i+1 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(2) 
print("==End of dump process==") 
file_object.write(driver.page_source)
driver.close()

Log in sucessfully
==End of dump process==


In [248]:
f=codecs.open("Dump_"+shopName+".txt", "r", "utf-8") 
source = f.read() 

soup = BeautifulSoup(source, 'html.parser') 


In [249]:
TAG_RE = re.compile(r'<[^>]+>') 

def display_review(review):
    for each_review in review:
        print(each_review,'\n')

def remove_tags(text): 
    return TAG_RE.sub('', text) 

def get_review_from_html(soup):
    data = soup.find_all("div", class_="_5pbx userContent _3576") 
    tag = ""
    bag_of_data = []
    for d in data: 
        tag = d.find("p") 
        tag = remove_tags(str(tag)) 
        bag_of_data.append(tag)
    return bag_of_data

#Get review of the restuarant from HTML5 file.
review = get_review_from_html(soup)
print("Reviews: {}".format(review))
print("Type of Reviews: {}".format(len(review)))
#print("Reviews: {}".format(review[:10]))
#display_review(review)

Reviews: ['Review only for takeaway, as I have never dined in. Great food at great prices! Service is also fast and professional. Will come back!', 'Large selection of pizza, pastas and desserts (wow). Southern Italy recipes not seen anywhere else. Italian beers on draft too. Very popular with locals and expats. Attentive staff. 5 stars!', 'อย่าไปซื้อพนังงานนิสัยเเย่มาด่าลูกค้าเหี้ย ผู้จัดการเข้าข้างลูก', 'Pala pizza 5 Stars  The best Roman pizza ever in Bangkok! fast and effective service. traditional and authentic Italian food !  thank you so much .', 'ดีทุกอย่าง...อาหาร. รสชาติ. สภานที่. บริการ... ไม่ต้องอธิบายมาก', 'It eat at Pala whenever I am in Bangkok. It is a great place and the food and service is excellent. I see the same faces there all the time so I am not alone in thinking that this is THE place for good pastas and pizzas and more.', 'อาหารสดใหม่ ชีสในพิซซ่าใช้ของดี ทานละใช่เลย', 'Great pizzeria restaurant located in the center of Bangkok. The pasta is really nice and I l

In [250]:
#Visualize เวลาที่โพสต์
#Get เวลา
time_data = soup.find_all("span", class_="timestampContent")
print("Length of Time_data: {}".format(len(time_data)))
year = ['2013', '2014', '2015', '2016', '2017']
result = []
count = 0

def preprocess_word(time_data, year):
    data = []
    for i in range(len(time_data)):
        #Remove tag of HTML 
        time_data[i] = remove_tags(str(time_data[i]))
        #Split string list by white space. 
        time_data[i] = time_data[i].split(' ')

        #Check whether white space in a word and then delete white space.
        if '' in time_data[i]:
            time_data[i].remove('')

        # Delete 'เวลา' in a word.
        if 'เวลา' in time_data[i]:
            time_data[i].remove('เวลา')

        # Delete 'น.' in a word.
        if 'น.' in time_data[i]:
            time_data[i].remove('น.')

        # If each row does not have a year in last index, Then add year '2017' to last index
        if len(time_data[i]) == 2:
            time_data[i].insert(3, '2017')

        #Check last index of each data is one of the '2014, 2015, 2016, 2017, 2018' or not 
        #If it does not, Then change the last index to '2018'
        if time_data[i][-1] not in year:
            time_data[i][-1] = '2017'
        
        data.append(time_data[i])
        
    return data

time_data = preprocess_word(time_data, year)
#Convert string list to string data type
print(time_data[0])
print(type(time_data))

Length of Time_data: 204
['7', 'กันยายน', '2017']
<class 'list'>


In [251]:
#Prepare time data to CSV format
date = []
month = []
year = []
for i in range(len(time_data)):
    date.append(int(time_data[i][0])) # For adding date to list
    month.append(time_data[i][1]) # For adding month to list
    year.append(int(time_data[i][2])) # For adding year to list
#print(month, year)

data = {'Date': date,
         'Month': month,
         'Year': year,
       }
df = pd.DataFrame.from_dict(data)
#Create excel writer
df.to_csv('csv_'+shopName+'.csv', encoding='utf-8', index=False)
print("Save sucessfully")

Save sucessfully


In [252]:
# sum = 1
# oldDateStr = None
# with open('mademoiselle.csv', newline='') as csvfile:
#     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
#     next(spamreader, None)
#     for row in spamreader:
#         dateStr = str(', '.join(row))
#         if(oldDateStr == None):
#             oldDateStr = dateStr
#             continue
#         if(dateStr != oldDateStr):
#             print(oldDateStr+' '+str(sum))
#             oldDateStr = dateStr
#             sum = 1
#         else:
#             sum += 1

In [253]:
monthno = {"มกราคม": 1,
            "กุมภาพันธ์": 2,
            "มีนาคม": 3,
            "เมษายน": 4,
            "พฤษภาคม": 5,
            "มิถุนายน": 6,
            "กรกฎาคม": 7,
            "สิงหาคม": 8,
            "กันยายน": 9,
            "ตุลาคม": 10,
            "พฤศจิกายน": 11,
            "ธันวาคม": 12}

sum = 1
oldMonthYearStr = None
ls_result = []
result_array = np.empty((4, 4))
with open('csv_'+shopName+'.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    next(spamreader, None)
    
    for row in spamreader:
        monthYear = str(','.join(row)).split(',')
        monthYear[1] = str(monthno[str(monthYear[1])])
        monthYearStr = str(','.join(monthYear))
        if(oldMonthYearStr == None):
            oldMonthYearStr = monthYearStr
            continue
        if(oldMonthYearStr != monthYearStr):
            result = oldMonthYearStr+','+str(sum)
            l = list(map(int, result.split(',')))
            ls_result.append(l)
#             print(oldMonthYearStr+','+str(sum))
            oldMonthYearStr = monthYearStr
            sum = 1
        else:
            sum += 1
print(ls_result)

a=np.array(ls_result)
a=a[a[:,0].sort()]
a=a[:,:,3]
a=np.asarray(a[0])
result=str(a).replace('\n','').replace('[','').replace(']','')
result = result.split(' ')
result = list(map(int, result))
result = str(', '.join(str(x) for x in result))

print(result)

[[7, 9, 2017, 1], [6, 9, 2017, 1], [28, 3, 2017, 1], [5, 9, 2017, 1], [7, 3, 2017, 1], [5, 9, 2017, 1], [3, 7, 2017, 1], [5, 9, 2017, 1], [11, 7, 2017, 1], [3, 9, 2017, 1], [30, 1, 2017, 1], [6, 9, 2017, 1], [10, 9, 2017, 1], [4, 9, 2017, 1], [8, 1, 2017, 1], [6, 9, 2017, 1], [17, 2, 2017, 1], [5, 9, 2017, 1], [8, 6, 2017, 1], [5, 9, 2017, 1], [5, 12, 2017, 1], [6, 9, 2017, 1], [3, 4, 2017, 1], [5, 9, 2017, 1], [10, 11, 2017, 1], [1, 11, 2017, 1], [22, 8, 2017, 1], [5, 9, 2017, 1], [4, 1, 2017, 1], [13, 9, 2017, 1], [11, 1, 2017, 1], [2, 1, 2017, 1], [22, 6, 2017, 1], [10, 2, 2017, 1], [6, 4, 2017, 1], [11, 7, 2017, 1], [12, 5, 2017, 1], [22, 11, 2017, 1], [1, 10, 2017, 1], [3, 12, 2017, 1], [4, 9, 2017, 1], [22, 10, 2017, 1], [19, 11, 2017, 1], [24, 10, 2017, 1], [6, 6, 2017, 1], [13, 8, 2017, 1], [5, 9, 2017, 1], [15, 11, 2017, 1], [4, 7, 2017, 1], [13, 8, 2017, 1], [17, 11, 2017, 1], [30, 8, 2017, 1], [7, 7, 2017, 1], [27, 5, 2017, 1], [1, 11, 2017, 1], [24, 9, 2017, 1], [7, 10, 201

In [254]:
file_object = codecs.open("result_"+shopName+".txt", "w", "utf-8")
file_object.write(result)
file_object.close()

In [260]:
dot_chart = pygal.Dot(x_label_rotation=30)

for shop in graphFileList:
    f=codecs.open("result_"+shop+".txt", "r", "utf-8") 
    summary = f.read() 
    l = list(map(int, summary.split(',')))
    l = l[:40]
    print(l)
    dot_chart.add(shop, l)

dot_chart.render_to_file('graph_result.svg') 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
